In [0]:
#Import GraphFrame Packages
from functools import reduce
from pyspark.sql.functions import col, lit, when
from graphframes import *

vertices = sqlContext.createDataFrame([
 ("a", "Alice", 34),
 ("b", "Bob", 36),
 ("c", "Charlie", 30),
 ("d", "David", 29),
 ("e", "Esther", 32),
 ("f", "Fanny", 36),
 ("g", "Gabby", 60)], ["id", "name", "age"])

In [0]:
pip install graphframes

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
edges = sqlContext.createDataFrame([
 ("a", "b", "friend"),
 ("b", "c", "follow"),
 ("c", "b", "follow"),
 ("f", "c", "follow"),
 ("e", "f", "follow"),
 ("e", "d", "friend"),
 ("d", "a", "friend"),
 ("a", "e", "friend")
], ["src", "dst", "relationship"])


In [0]:
g = GraphFrame("vertices","edges")
print(g)

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<command-4052307809996932> in <module>
----> 1 g = GraphFrame("vertices","edges")
      2 print(g)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-bee0bbcb-4a07-4236-b17f-0905570ccdab/lib/python3.8/site-packages/graphframes/graphframe.py in __init__(self, v, e)
     61         self._vertices = v
     62         self._edges = e
---> 63         self._sqlContext = v.sql_ctx
     64         self._sc = self._sqlContext._sc
     65         self._jvm_gf_api = _java_api(self._sc)

AttributeError: 'str' object has no attribute 'sql_ctx'

In [0]:
# This example graph also comes with the GraphFrames package.
from graphframes.examples import Graphs
same_g = Graphs(sqlContext).friends()
print(same_g)



---------------------------------------------------------------------------
ImportError                               Traceback (most recent call last)
<command-4052307809996933> in <module>
      1 # This example graph also comes with the GraphFrames package.
----> 2 from graphframes.examples import Graphs
      3 same_g = Graphs(sqlContext).friends()
      4 print(same_g)
      5 

/databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py in import_patch(name, globals, locals, fromlist, level)
    165             # Import the desired module. If you’re seeing this while debugging a failed import,
    166             # look at preceding stack frames for relevant error information.
--> 167             original_result = python_builtin_import(name, globals, locals, fromlist, level)
    168 
    169             is_root_import = thread_local._nest_level == 1

/local_disk0/.ephemeral_nfs/envs/pythonEnv-bee0bbcb-4a07-4236-b17f-0905570ccdab/lib/python3.8/site-packages/

In [0]:
#Display List of Vertices
display(g.vertices)


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-4052307809996934> in <module>
      1 #Display List of Vertices
----> 2 display(g.vertices)

NameError: name 'g' is not defined

In [0]:
#Display Edges
display(g.edges)


In [0]:
#Display the incoming degree of the vertices
display(g.inDegrees)


In [0]:
#Display the outgoing degree of the vertices:
display(g.outDegrees)


In [0]:
#Display the degree of vertices
display(g.degrees)


In [0]:
#we can run queries directly on the vertices DataFrame.
#For example, we can find the age of the youngest person in the graph:
youngest = g.vertices.groupBy().min("age")
display(youngest)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-4052307809996939> in <module>
      1 #we can run queries directly on the vertices DataFrame.
      2 #For example, we can find the age of the youngest person in the graph:
----> 3 youngest = g.vertices.groupBy().min("age")
      4 display(youngest)

NameError: name 'g' is not defined

In [0]:
#Likewise, we can run queries on the edges of DataFrame.
#For example, let's count the number of 'follow' relationships in the graph:

numFollows = g.edges.filter("relationship = 'follow'").count()
print("The number of follow edges is", numFollows)

numFollows = g.edges.filter("relationship = 'friend'").count()
print("The number of friends edges is", numFollows)

motifs = g.find("(a)-[e]->(b); (b)-[e2]->(a)")
display(motifs)

filtered = motifs.filter("b.age > 30 or a.age > 30")


In [0]:
# Find chains of 4 vertices.
chain4 = g.find("(a)-[ab]->(b); (b)-[bc]->(c); (c)-[cd]->(d)")
display(chain4)



In [0]:
# Query on sequence, with state (cnt)
#  (a) Define method for updating state given the next element of the motif.
def cumFriends(cnt, edge):
    relationship = col(edge)["relationship"]
return when(relationship == "friend", cnt + 1).otherwise(cnt)


  File "<command-4052307809996942>", line 5
    return when(relationship == "friend", cnt + 1).otherwise(cnt)
    ^
SyntaxError: 'return' outside function


In [0]:
#(b) Use sequence operation to apply method to sequence of elements in motif.
#   In this case, the elements are the 3 edges.
edges = ["ab", "bc", "cd"]
numFriends = reduce(cumFriends, edges, lit(0))


chainWith2Friends2 = chain4.withColumn("num_friends", numFriends).where(numFriends >= 2)
display(chainWith2Friends2)

g2 = g.filterEdges("relationship = 'friend'").filterVertices("age > 30").dropIsolatedVertices()

display(g2.vertices)
display(g2.edges)

